In [1]:
from ria import ria

print('OK !')

OK !


In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from pythainlp.tokenize import word_tokenize
from string import punctuation
import numpy as np

import time

def reset(df):
    cols = df.columns
    return df.reset_index()[cols]

print('OK !')

OK !


In [3]:
def canonicalize(string):
    normalized_tokens = list()
    a = word_tokenize(string, engine = 'newmm')
    for j in a:
        token = j.strip()
        #Add clean statement here 
        if len(token) > 1 and token not in set(punctuation) and token not in ['..','...','ๆๆ']:
            try:
                normalized_tokens.append(token.lower())
            except:
                normalized_tokens.append(token)
                pass
    return normalized_tokens

def query(query_text, vectorizer, document_matrix, df):
    
    # Transform our string using the vocabulary
    transformed = vectorizer.transform([query_text])
    query = transformed[0:1]

    np_result = np.array(find_similar(query, document_matrix, len(df)))

    mask = np_result[:, 1] > 0 

    np_result = np_result[mask, :]
        
    index_list = []
    score_list = []
    df_dict = {'index': index_list, 'score': score_list}

    df_dict['index'] =  np_result[:,0]
    df_dict['score'] =  np_result[:,1]

    df = pd.DataFrame(df_dict)
    df['index'] = df['index'].astype(int)
    df = df.set_index('index')
    
    return reset(pd.merge(df, data, left_index=True, right_index=True, how ='left'))


# A function that given an input query item returns the top-k most similar items 
# by their cosine similarity.
def find_similar(query_vector, td_matrix, top_k = 5):
    cosine_similarities = cosine_similarity(query_vector, td_matrix).flatten()
    related_doc_indices = cosine_similarities.argsort()[::-1]
    return [(index, cosine_similarities[index]) for index in related_doc_indices][0:top_k]

print('OK !')

OK !


In [4]:
app = ria()

In [5]:
data = app.df_meta_question_answer.copy()

In [6]:
# tfidf_vectorizer = TfidfVectorizer(tokenizer=canonicalize, sublinear_tf=True)

# #This can take time; after finnishing this, you can save transformed document into pickle file for later usage
# start = time.time()

# tfidf_term_document_matrix = tfidf_vectorizer.fit_transform(data['Original_text'])

# print("Took " + "%.2f" % float(time.time() - start) + " sec")

In [7]:
# #dump pickle files
# import pickle
# with open('df_list.pickle', 'wb') as handle:
#     pickle.dump(df_list, handle)

# with open('df_list.pickle', 'rb') as handle:
#     df_list = pickle.load(handle)

In [8]:
#dump pickle files
# import pickle
# with open('tfidf_vectorizer.pickle', 'wb') as handle:
#     pickle.dump(tfidf_vectorizer, handle)

In [9]:
import pickle

with open('09_Output_Streamlib/tfidf_vectorizer.pickle', 'rb') as handle:
    tfidf_vectorizer = pickle.load(handle)
with open('09_Output_Streamlib/tfidf_term_document_matrix.pickle', 'rb') as handle:
    tfidf_term_document_matrix = pickle.load(handle)

In [10]:
tfidf_term_document_matrix

<2954x10040 sparse matrix of type '<class 'numpy.float64'>'
	with 348463 stored elements in Compressed Sparse Row format>

In [11]:
query_text = "บริษัท"

result_df = query(query_text, tfidf_vectorizer, tfidf_term_document_matrix, data)

result_df

,score,Doc_Page_ID,Original_text,Doc_ID,Page_ID,File_Name,เรื่อง,File_Code,Category_Code,หัวเรื่อง,ประเภทเอกสาร,วันที่สิ้นสุด,สถานะ,สถาบันผู้เกี่ยวข้อง,กฎหมาย,เลขที่ (Thai)
0,0.272131,0000|0050,ทั้งนี้ การตัดสินใจเลือกรูปแบบของระบบเทคโนโลยี...,0,50,สนส. 27/2551 การอนุญาตให้บริษัทเงินทุนและบริษั...,การอนุญาตให้บริษัทเงินทุนและบริษัทเครดิตฟองซิเ...,25510317.pdf,1,['สถาบันการเงิน'],['ประกาศ ธปท.'],['39664'],['ใช้อยู่'],"['บริษัทเงินทุน', 'บริษัทเครดิตฟองซิเอร์']",['พรบ.สง. มาตรา 36'],สนส.27/2551
1,0.225576,0068|0032,รายชื่อบริษัทในกลุ่ม\nรายได้ดอกเบี้ย\nรายได้มิ...,68,32,นำส่งแบบรายงานการกำกับแบบรวมกลุ่ม,นำส่งแบบรายงานการกำกับแบบรวมกลุ่ม,25530155.pdf,4,['สถาบันการเงิน'],['หนังสือเวียน'],['1'],['ใช้อยู่'],"['ธนาคารพาณิชย์จดทะเบียนในประเทศ', 'บริษัทเงิน...",['พรบ.สง. มาตรา 71'],ฝนส.(23)ว.76/2553
2,0.222883,0096|0010,กลุ่มธุรกิจทางการเงินที่บริษัทแม่เป็นบริษัทเคร...,96,10,นำส่งหลักเกณฑ์การกำกับดูแลโครงสร้างและขอบเขตธุ...,นำส่งหลักเกณฑ์การกำกับดูแลโครงสร้างและขอบเขตธุ...,25640142.pdf,4,['สถาบันการเงิน'],"['ประกาศ ธปท.', 'หนังสือเวียน']",['2021-08-13 00:00:00'],['ใช้อยู่'],['บริษัทเครดิตฟองซิเอร์'],"['พรบ.สง. มาตรา 57', 'พรบ.สง. มาตรา 54', 'พร...",สนส. 7/2564
3,0.215188,0023|0037,บริษัท B\nถือหุ้นเกิน 50%\nให้สินเชื่อ\n(3.4) ...,23,37,หลักเกณฑ์การกำกับลูกหนี้้ีรายใหญ่ (Single Lend...,หลักเกณฑ์การกำกับลูกหนี้รายใหญ่ (Single Lendin...,25550350.pdf,2,['สถาบันการเงิน'],"['หนังสือเวียน', 'ประกาศ ธปท.']",['41275'],['ใช้อยู่'],"['ธนาคารพาณิชย์จดทะเบียนในประเทศ', 'บริษัทเงิน...","['พรบ.สง. มาตรา 50', 'พรบ.สง. มาตรา 51']",สนส. 22/2555
4,0.213155,0068|0008,ลำดับที\nบริษัทในกลุ่มธุรกิจทางการเงิน\nธนาคาร...,68,8,นำส่งแบบรายงานการกำกับแบบรวมกลุ่ม,นำส่งแบบรายงานการกำกับแบบรวมกลุ่ม,25530155.pdf,4,['สถาบันการเงิน'],['หนังสือเวียน'],['1'],['ใช้อยู่'],"['ธนาคารพาณิชย์จดทะเบียนในประเทศ', 'บริษัทเงิน...",['พรบ.สง. มาตรา 71'],ฝนส.(23)ว.76/2553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,0.026643,0066|0044,ประเด็นคำถาม - คำตอบสำหรับประกาศ ธปท. เรื่อง ก...,66,44,หลักเกณฑ์การดำรงเงินกองทุนขั้นต่ำสำหรับความเสี...,หลักเกณฑ์การดำรงเงินกองทุนขั้นต่ำสำหรับความเสี...,25510555.pdf,3,['สถาบันการเงิน'],['ประกาศ ธปท.'],['39813'],['ใช้อยู่'],"['ธนาคารพาณิชย์จดทะเบียนในประเทศ', 'ธนาคารพาณิ...",[],สนส. 95/2551
793,0.026587,0069|0007,22. ประมาณการหนีสิน\n22.1 ประมาณการหนี้สินสำหร...,69,7,นำส่งแบบรายงานการกำกับแบบรวมกลุ่มที่แก้ไขเพิ่ม...,นำส่งแบบรายงานการกำกับแบบรวมกลุ่มที่แก้ไขเพิ่ม...,25540002.pdf,4,['สถาบันการเงิน'],['หนังสือเวียน'],['1'],['ใช้อยู่'],"['ธนาคารพาณิชย์จดทะเบียนในประเทศ', 'บริษัทเงิน...",['พรบ.สง. มาตรา 71'],ฝนส. (23) ว. 1/2554
794,0.026131,0070|0082,ตารางที่ 2 : การคำนวณเงินกองทุนขั้นต่ำสำหรับคว...,70,82,หลักเกณฑ์การดำรงเงินกองทุนขั้นต่ำสำหรับความเสี...,หลักเกณฑ์การดำรงเงินกองทุนขั้นต่ำสำหรับความเสี...,25550082.pdf,3,['สถาบันการเงิน'],['ประกาศ ธปท.'],['1'],['ใช้อยู่'],"['ธนาคารพาณิชย์จดทะเบียนในประเทศ', 'ธนาคารพาณิ...","['พรบ.สง. มาตรา 29', 'พรบ.สง. มาตรา 30', 'พร...",สนส.5/2555
795,0.020994,0075|0058,ระบุชื่อกิจการตามกฎหมายที่เป็นผู้ออกตราสารทางก...,75,58,นำส่งประกาศธนาคารแห่งประเทศไทย เรื่อง การเปิดเ...,นำส่งประกาศธนาคารแห่งประเทศไทย เรื่อง การเปิดเ...,25560150.pdf,3,['สถาบันการเงิน'],"['ประกาศ ธปท.', 'หนังสือเวียน']",['1'],['ใช้อยู่'],"['ธนาคารพาณิชย์จดทะเบียนในประเทศ', 'ธนาคารพาณิ...",['พรบ.สง. มาตรา 31'],สนส. 4/2556
